In [1]:
!pip install qiskit --quiet


In [2]:
!pip uninstall -y qiskit
!pip install qiskit==0.45.0 qiskit-optimization==0.6.0


Found existing installation: qiskit 0.45.0
Uninstalling qiskit-0.45.0:
  Successfully uninstalled qiskit-0.45.0
  Using cached qiskit-0.45.0-py3-none-any.whl.metadata (12 kB)
Using cached qiskit-0.45.0-py3-none-any.whl (9.6 kB)


In [3]:
from google.colab import files
uploaded = files.upload()  # Upload your synthetic_graph.graphml file


Saving synthetic_graph.graphml to synthetic_graph (3).graphml


In [4]:
import networkx as nx

# Load your graph
G = nx.read_graphml("synthetic_graph.graphml")

# Extract a small agent-task subgraph (just 3 relationships)
subG = nx.Graph()
task_count = 0

for u, v, d in G.edges(data=True):
    if d.get("relation") == "PERFORMS" and G.nodes[u]["type"] == "Agent" and G.nodes[v]["type"] == "Task":
        subG.add_edge(u, v, **d)
        subG.nodes[u]["type"] = "Agent"
        subG.nodes[v]["type"] = "Task"
        task_count += 1
        if task_count >= 3:
            break

# View the nodes and edges
print("Subgraph nodes:", subG.nodes(data=True))
print("Subgraph edges:", subG.edges(data=True))


Subgraph nodes: [('agent_0', {'type': 'Agent'}), ('task_0', {'type': 'Task'}), ('agent_1', {'type': 'Agent'}), ('task_1', {'type': 'Task'}), ('agent_2', {'type': 'Agent'}), ('task_2', {'type': 'Task'})]
Subgraph edges: [('agent_0', 'task_0', {'relation': 'PERFORMS'}), ('agent_1', 'task_1', {'relation': 'PERFORMS'}), ('agent_2', 'task_2', {'relation': 'PERFORMS'})]


In [5]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import QAOA
from qiskit.primitives import Estimator
from qiskit.utils import algorithm_globals
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model

# Step 1: Create a model
mdl = Model(name="task_assignment")

# Step 2: Create binary variables: x_ij = 1 if agent i performs task j
agents = sorted([n for n, d in subG.nodes(data=True) if d["type"] == "Agent"])
tasks = sorted([n for n, d in subG.nodes(data=True) if d["type"] == "Task"])
x = {(i, j): mdl.binary_var(name=f"x_{i}_{j}") for i in agents for j in tasks}

# Step 3: Objective — maximize assignments (dummy value of 1)
mdl.maximize(mdl.sum(x[i, j] for i in agents for j in tasks if subG.has_edge(i, j)))

# Step 4: Constraints
for i in agents:
    mdl.add_constraint(mdl.sum(x[i, j] for j in tasks if subG.has_edge(i, j)) <= 1)

for j in tasks:
    mdl.add_constraint(mdl.sum(x[i, j] for i in agents if subG.has_edge(i, j)) <= 1)

# Step 5: Convert to QUBO
qp = from_docplex_mp(mdl)
qubo = QuadraticProgramToQubo().convert(qp)
print("QUBO created:")
print(qubo.export_as_lp_string())


QUBO created:
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: task_assignment

Minimize
 obj: - 17 x_agent_0_task_0 - 17 x_agent_1_task_1 - 17 x_agent_2_task_2
      - 8 c0@int_slack@0 - 8 c1@int_slack@0 - 8 c2@int_slack@0
      - 8 c3@int_slack@0 - 8 c4@int_slack@0 - 8 c5@int_slack@0 + [
      16 x_agent_0_task_0^2 + 16 x_agent_0_task_0*c0@int_slack@0
      + 16 x_agent_0_task_0*c3@int_slack@0 + 16 x_agent_1_task_1^2
      + 16 x_agent_1_task_1*c1@int_slack@0 + 16 x_agent_1_task_1*c4@int_slack@0
      + 16 x_agent_2_task_2^2 + 16 x_agent_2_task_2*c2@int_slack@0
      + 16 x_agent_2_task_2*c5@int_slack@0 + 8 c0@int_slack@0^2
      + 8 c1@int_slack@0^2 + 8 c2@int_slack@0^2 + 8 c3@int_slack@0^2
      + 8 c4@int_slack@0^2 + 8 c5@int_slack@0^2 ]/2 + 24
Subject To

Bounds
 0 <= x_agent_0_task_0 <= 1
 0 <= x_agent_0_task_1 <= 1
 0 <= x_agent_0_task_2 <= 1
 0 <= x_agent_1_task_0 <= 1
 0 <= x_agent_1_task_1 <= 1
 0 <= x_agent_1_task_2 <= 1
 0 <= x_agent_2_task_0 

/tmp/ipython-input-5-3870513390.py:3: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms import QAOA
